### 00. Importamos librerías

In [311]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

### 01. Elegimos ubicaciones

Como sabemos que todos los empleados son jóvenes y les gusta salir, hemos decidido situar como punto estratégico unicaciones de sitios emblemáticos en 3 capitales europeas.

- Mad_Puerta_del_sol = [40.41706973493005, -3.703560607026817]
- Roma_Coliseo = [41.89020219240534, 12.492187981868335]
- Berlin_Puerta_Brandenburgo = [52.51641823520535, 13.377618269311524]


In [316]:
m = {"latt" : 40.41706973493005, "longt": -3.703560607026817} #madrid 
r = {"latt" : 41.89020219240534, "longt" :12.492187981868335} #roma
b = {"latt" : 52.51641823520535, "longt" :13.377618269311524} #berlín

Obtenemos el tipo Point de nuestras ubicaciones

In [317]:
def devuelvetipoPoint(x):
    return {"type": "Point", "coordinates": [ x["longt"],x["latt"]]}

In [318]:
m = devuelvetipoPoint(m)

In [319]:
r = devuelvetipoPoint(r)

In [320]:
b = devuelvetipoPoint(b)

### 02. Vamos a empezar a hacer queries a la API

In [321]:
load_dotenv()

True

In [322]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [324]:
url_query = 'https://api.foursquare.com/v2/venues/search'

Hemos elegido como criterios fundamentales, Starbucks, veterinarios y Aeropuertos, ahora vamos a hacer una query por cada criterio

Starbucks Madrid

In [325]:
parametros_m_star = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{m['coordinates'][1]}, {m['coordinates'][0]}",
    "query": "Starbucks"
}

In [326]:
resp_m_star = requests.get(url_query, params = parametros_m_star).json()

In [327]:
m_star = list(resp_m_star["response"]["venues"])


Starbucks Roma

In [328]:
parametros_r_star = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{r['coordinates'][1]}, {r['coordinates'][0]}",
    "query": "Starbucks"
}

In [329]:
resp_r_star = requests.get(url_query, params = parametros_r_star).json()

In [330]:
r_star = list(resp_r_star["response"]["venues"])
len(r_star)

2

Starbucks en Berlín

In [331]:
parametros_b_star = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{b['coordinates'][1]}, {b['coordinates'][0]}",
    "query": "Starbucks"
}

In [332]:
resp_b_star = requests.get(url_query, params = parametros_b_star).json()

In [333]:
b_star = list(resp_b_star["response"]["venues"])
len(b_star)

20

Veterinario en Madrid

In [334]:
parametros_m_dobby = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{m['coordinates'][1]}, {m['coordinates'][0]}",
    "query": "Vet"
}

In [335]:
resp_m_dobby = requests.get(url_query, params = parametros_m_dobby).json()

In [336]:
m_dobby = list(resp_m_dobby["response"]["venues"])

Veterinario en Roma

In [337]:
parametros_r_dobby = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{r['coordinates'][1]}, {r['coordinates'][0]}",
    "query": "Vet"
}

In [338]:
resp_r_dobby = requests.get(url_query, params = parametros_r_dobby).json()

In [339]:
r_dobby = list(resp_r_dobby["response"]["venues"])

Vererinario en Berlín

In [340]:
parametros_b_dobby = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{b['coordinates'][1]}, {b['coordinates'][0]}",
    "query": "Vet"
}

In [341]:
resp_b_dobby = requests.get(url_query, params = parametros_b_dobby).json()

In [342]:
b_dobby = list(resp_b_dobby["response"]["venues"])

Aeropuerto en Madrid

In [343]:
parametros_m_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{m['coordinates'][1]}, {m['coordinates'][0]}",
    "query": "Airport"
}

In [344]:
resp_m_air = requests.get(url_query, params = parametros_m_air).json()

In [345]:
m_air = list(resp_m_air["response"]["venues"])

Aeropuerto en Roma

In [346]:
parametros_r_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{r['coordinates'][1]}, {r['coordinates'][0]}",
    "query": "Airport"
}

In [347]:
resp_r_air = requests.get(url_query, params = parametros_r_air).json()

In [348]:
r_air = list(resp_r_air["response"]["venues"])

In [349]:
len(list(resp_r_air))

2

Aeropuerto en Berlin

In [350]:
parametros_b_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{b['coordinates'][1]}, {b['coordinates'][0]}",
    "query": "Airport"
}

In [351]:
resp_b_air = requests.get(url_query, params = parametros_b_air).json()

In [352]:
b_air = list(resp_b_air["response"]["venues"])

In [353]:
len(list(resp_b_air))

2

### 03. Limpiamos la respuesta de la API -- generamos dataframes limpitos

#### Definimos funciones de limpieza

In [142]:
def esmadrid(x):
    return "madrid"

In [143]:
def esroma(x):
    return "roma"

In [144]:
def esberlin(x):
    return "berlin"

In [145]:
def esstar(x):
    return "starbucks"

In [146]:
def esvet(x):
    return "vet"

In [147]:
def esairport(x):
    return "airport"

In [150]:
def devuelvedistancia(x):
    return x["distance"]

In [152]:
def devuelvePoint(x):
    return {"type": "Point", "coordinates": [x["lng"], x["lat"]]}

#### Sacamos el dataframe de Starbucks de Madrid

In [354]:
df_m_star = pd.DataFrame(m_star)

In [141]:
df_m_star.drop(["id", "contact", "categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk" ], axis=1, inplace=True)

In [148]:
df_m_star["ciudad"] = df_m_star["name"].apply(esmadrid)

In [149]:
df_m_star["criterio"] = df_m_star["name"].apply(esstar)

In [151]:
df_m_star["distancia"] = df_m_star["location"].apply(devuelvedistancia)

In [153]:
df_m_star["point"] = df_m_star["location"].apply(devuelvePoint)

#### Sacamos el dataframe de Vet de Madrid 

In [154]:
df_m_dobby = pd.DataFrame(m_dobby)

In [155]:
df_m_dobby.drop(["id","venuePage", "contact", "categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk"], axis=1, inplace=True)

In [156]:
df_m_dobby["ciudad"] = df_m_dobby["name"].apply(esmadrid)

In [157]:
df_m_dobby["criterio"] = df_m_dobby["name"].apply(esvet)

In [158]:
df_m_dobby["distancia"] = df_m_dobby["location"].apply(devuelvedistancia)

In [159]:
df_m_dobby["point"] = df_m_dobby["location"].apply(devuelvePoint)

#### Sacamos el dataframe de Aeropuerto de Madrid 

In [160]:
df_m_air = pd.DataFrame(m_air)

In [161]:
df_m_air.drop(["id", "contact", "venuePage","categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk" ], axis=1, inplace=True)

In [162]:
df_m_air["distancia"] = df_m_air["location"].apply(devuelvedistancia)

In [163]:
df_m_air["ciudad"] = df_m_air["name"].apply(esmadrid)

In [164]:
df_m_air["criterio"] = df_m_air["name"].apply(esairport)

In [165]:
df_m_air["point"] = df_m_air["location"].apply(devuelvePoint)

#### Sacamos el dataframe deStarbucks de Roma 

In [166]:
df_r_star = pd.DataFrame(r_star)

In [167]:
df_r_star.drop(["id", "contact", "categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk" ], axis=1, inplace=True)

In [168]:
df_r_star["distancia"] = df_r_star["location"].apply(devuelvedistancia)

In [169]:
df_r_star["ciudad"] = df_r_star["name"].apply(esroma)

In [170]:
df_r_star["criterio"] = df_r_star["name"].apply(esstar)

In [171]:
df_r_star["point"] = df_r_star["location"].apply(devuelvePoint)

#### Sacamos el dataframe de Vets en Roma 

In [172]:
df_r_dobby = pd.DataFrame(r_dobby)

In [173]:
df_r_dobby.drop(["id", "contact", "categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk" ], axis=1, inplace=True)

In [174]:
df_r_dobby["distancia"] = df_r_dobby["location"].apply(devuelvedistancia)

In [175]:
df_r_dobby["ciudad"] = df_r_dobby["name"].apply(esroma)

In [176]:
df_r_dobby["criterio"] = df_m_dobby["name"].apply(esvet)

In [177]:
df_r_dobby["point"] = df_r_dobby["location"].apply(devuelvePoint)

#### Sacamos el dataframe de Aeropuertos de Roma 

In [178]:
df_r_air = pd.DataFrame(r_air)

In [179]:
df_r_air.drop(["id", "contact", "venuePage","categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk"], axis=1, inplace=True)

In [180]:
df_r_air["distancia"] = df_r_air["location"].apply(devuelvedistancia)

In [181]:
df_r_air["ciudad"] = df_r_air["name"].apply(esroma)

In [182]:
df_r_air["criterio"] = df_r_air["name"].apply(esairport)

In [183]:
df_r_air["point"] = df_r_air["location"].apply(devuelvePoint)

#### Sacamos el dataframe deStarbucks de Berlin 

In [184]:
df_b_star = pd.DataFrame(b_star)

In [185]:
df_b_star.drop(["id", "contact", "categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk"], axis=1, inplace=True)

In [186]:
df_b_star["ciudad"] = df_b_star["name"].apply(esberlin)

In [187]:
df_b_star["criterio"] = df_b_star["name"].apply(esstar)

In [188]:
df_b_star["distancia"] = df_b_star["location"].apply(devuelvedistancia)

In [189]:
df_b_star["point"] = df_b_star["location"].apply(devuelvePoint)

#### Sacamos el dataframe de Vets de Berlin 

In [355]:
df_b_dobby = pd.DataFrame(b_dobby)

In [356]:
df_b_dobby.drop(["id","venuePage", "contact", "categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk"], axis=1, inplace=True)

In [357]:
df_b_dobby["ciudad"] = df_m_dobby["name"].apply(esberlin)

In [358]:
df_b_dobby["criterio"] = df_b_dobby["name"].apply(esvet)

In [359]:
df_b_dobby["distancia"] = df_b_dobby["location"].apply(devuelvedistancia)

In [195]:
df_b_dobby["point"] = df_b_dobby["location"].apply(devuelvePoint)

#### Sacamos el dataframe de Aeropuestos de Berlin 

In [196]:
df_b_air = pd.DataFrame(b_air)

In [197]:
df_b_air.drop(["id", "contact", "venuePage","categories","verified","stats","beenHere","hereNow","referralId","venueChains", "hasPerk"], axis=1, inplace=True)

In [198]:
df_b_air["distancia"] = df_b_air["location"].apply(devuelvedistancia)

In [199]:
df_b_air["ciudad"] = df_b_air["name"].apply(esberlin)

In [200]:
df_b_air["criterio"] = df_b_air["name"].apply(esairport)

In [201]:
df_b_air["point"] = df_b_air["location"].apply(devuelvePoint)

#### Unimos toda la información de cada ciudad

In [203]:
client = MongoClient("localhost:27017")

In [204]:
db = client.get_database("Geoespecial")

In [205]:
madrid = pd.concat([df_m_star, df_m_dobby, df_m_air])
roma = pd.concat([df_r_star, df_r_dobby, df_r_air])
berlin = pd.concat([df_b_star, df_b_dobby, df_b_air])

In [206]:
data_mad =  madrid.to_dict(orient = "records")
client = MongoClient("localhost:27017")
db = client.get_database("Geoespecial")
madrid = db["Madrid"]
madrid.insert_many(data_mad)

In [207]:
data_rom =  roma.to_dict(orient = "records")
client = MongoClient("localhost:27017")
db = client.get_database("Geoespecial")
roma = db["Roma"]
roma.insert_many(data_rom)

In [208]:
data_ber =  berlin.to_dict(orient = "records")
client = MongoClient("localhost:27017")
db = client.get_database("Geoespecial")
berlin = db["Berlin"]
berlin.insert_many(data_ber)

### 04. Le damos peso a los criterios definidos y hacemos geoqueris

#### Le damos peso

In [288]:
def devuelvepuntos(x):
    if x == "starbucks":
        return 5 
    elif x == "vet":
        return 1
    else:
        return 10

#### hacemos las geoqueries y aplicamos los pesos de nuestros criterios para tener un dato por ciudad

In [361]:
mad = db.get_collection("Madrid")

In [362]:
mquery = [{'$geoNear':{'near': [m["coordinates"][0], m["coordinates"][1]],
                      'distanceField': 'd',
                      'maxDistance' : 1000,
                      'distanceMultiplier': 6371,
                      'spherical': True}}]
mad_geoloc = mad.aggregate(mquery)
             
pd_madrid = pd.DataFrame(list(mad_geoloc))

In [363]:
dfmadrid = pd.DataFrame(pd_madrid.groupby(pd_madrid["criterio"])["d"].mean().reset_index())

In [364]:
dfmadrid["puntos"] = dfromaaaa["criterio"].apply(devuelvepuntos)

In [365]:
dfmadrid["total"] = dfmadrid["puntos"]*  dfmadrid["d"]

In [366]:
TotalMADRID = dfmadrid['total'].sum()
print(TotalMADRID)

115.2135361279468


In [367]:
rom = db.get_collection("Roma")

In [368]:
rquery = [{'$geoNear':{'near': [r["coordinates"][0], r["coordinates"][1]],
                      'distanceField': 'd',
                      'maxDistance' : 1000,
                      'distanceMultiplier': 6371,
                      'spherical': True}}]
rom_geoloc = rom.aggregate(rquery)
             
pd_roma = pd.DataFrame(list(rom_geoloc))

In [369]:
dfromaaaa = pd.DataFrame(pd_roma.groupby(pd_roma["criterio"])["d"].mean().reset_index())

In [370]:
dfromaaaa["puntos"] = dfromaaaa["criterio"].apply(devuelvepuntos)

In [371]:
dfromaaaa["total"] = dfromaaaa["puntos"]*  dfromaaaa["d"]

In [372]:
TotalROMA = dfromaaaa['total'].sum()
print(TotalROMA)

126.45028613744243


In [373]:
dfromaaaa

,criterio,d,puntos,total
0,airport,10.352004,10,103.520044
1,starbucks,4.095577,5,20.477885
2,vet,2.452358,1,2.452358


In [374]:
ber = db.get_collection("Berlin")

In [375]:
bquery = [{'$geoNear':{'near': [b["coordinates"][0], b["coordinates"][1]],
                      'distanceField': 'd',
                      'maxDistance' : 1000,
                      'distanceMultiplier': 6371,
                      'spherical': True}}]

ber_geoloc = ber.aggregate(bquery)
pd_berlin = pd.DataFrame(list(ber_geoloc))

In [376]:
pd_berlin.groupby(pd_berlin["criterio"])["d"].mean()

criterio
airport      9.490385
starbucks    3.345740
vet          4.995744
Name: d, dtype: float64

In [377]:
dfberlin = pd.DataFrame(pd_berlin.groupby(pd_berlin["criterio"])["d"].mean().reset_index())
dfberlin

,criterio,d
0,airport,9.490385
1,starbucks,3.345740
2,vet,4.995744


In [378]:
dfberlin["puntos"] = dfberlin["criterio"].apply(devuelvepuntos)

In [379]:
dfberlin["total"] = dfberlin["puntos"]*  dfberlin["d"]
dfberlin

,criterio,d,puntos,total
0,airport,9.490385,10,94.903854
1,starbucks,3.345740,5,16.728702
2,vet,4.995744,1,4.995744


In [380]:
TotalBERLIN = dfberlin['total'].sum()
print(TotalBERLIN)

116.62829958061778


In [381]:
list([TotalMADRID,TotalROMA, TotalBERLIN] )

[115.2135361279468, 126.45028613744243, 116.62829958061778]

In [382]:
min([TotalMADRID,TotalROMA, TotalBERLIN] )

115.2135361279468

# El total más bajo y por tanto, las ubicaciones más cercanas a los puntos elegidos, pertenecen a MADRID

In [ ]:
m = {"latt" : 40.41706973493005, "longt": -3.703560607026817} #madrid 

In [384]:
inicial_lat = 40.41706973493005
inicial_long = -3.703560607026817

In [385]:
map_1 = folium.Map(location=[inicial_lat,inicial_long], zoom_start=15)
map_1